In [1]:
import pandas as pd
import glob
import os
import warnings
from datetime import datetime

warnings.filterwarnings("ignore")

In [2]:
file_pattern = "zfir60/ZFIR60_*.xlsx"
file_list = glob.glob(file_pattern)
# Lista vacía para almacenar los df individuales
dataframes = []

In [3]:
desde = "20240520"                          # Fecha de inicio
hasta = datetime.now().strftime("%Y%m%d")   # Fecha actual

fecha_hasta = datetime.strptime(hasta, '%Y%m%d')
fecha_desde = datetime.strptime(desde, '%Y%m%d')

In [4]:
def convertir_a_numero(text):
    return pd.to_numeric(text.replace(',', ''), errors='coerce')

In [5]:
def extraer_fecha_nombre_zfir(filename):
    basename = os.path.basename(filename)
    date_str = basename.split('_')[1].split('.')[0]
    return datetime.strptime(date_str, '%Y%m%d')

In [6]:
for file in file_list:
    fecha = extraer_fecha_nombre_zfir(file)
    if fecha_hasta >= fecha >= fecha_desde:
        df = pd.read_excel(file, usecols=['Cliente Pa', 'Garantia'])
        df = df.rename(columns={'Cliente Pa': 'Deudor'})
        df['Garantia'] = df['Garantia'].apply(convertir_a_numero)
        df = df.groupby('Deudor', as_index=False)['Garantia'].sum()
        df['Fecha'] = fecha
        df['Fecha'] = df['Fecha'].dt.strftime('%d/%m/%Y')
        df = df[['Fecha', 'Deudor', 'Garantia']]
        dataframes.append(df)

In [7]:
if dataframes:
    df_final = pd.concat(dataframes, ignore_index=True)
    df_final = df_final[df_final['Garantia'] != 0] # Elimina las garantias que sean 0
    df_final.to_excel(f'zfir60_{desde}_{hasta}.xlsx', index=False)
    os.startfile(f'zfir60_{desde}_{hasta}.xlsx')
else:
    print("No se encontraron archivos para procesar")